# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [5]:
import requests
import pandas as pd

## Список объектов

In [10]:
offset_ = 0
limit_ = 5000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data = res.json()
objects_data.get('data').get('list')[0]

{'objId': 29679,
 'developer': {'devId': 7424,
  'shortName': '212-2',
  'fullName': '212-2',
  'orgForm': {'id': 20,
   'fullForm': 'Жилищный или жилищно-строительный кооператив',
   'shortForm': 'ЖСК'},
  'devInn': '6154572922'},
 'rpdRegionCd': 61,
 'objAddr': 'г Таганрог',
 'objElemLivingCnt': 18,
 'objReady100PercDt': '2020-03-31',
 'objSquareLiving': 1519.0,
 'objCompensFundFlg': 0,
 'objProblemFlg': 0,
 'latitude': 47.2238,
 'longitude': 38.8513,
 'problemFlag': 'NONE',
 'siteStatus': 'Строящиеся',
 'objGuarantyEscrowFlg': 0,
 'residentialBuildings': 1,
 'rpdNum': '61-000818',
 'pdId': 124647,
 'buildType': 'Жилое',
 'objStatus': 0}

In [11]:
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list]
objids[:10]

[29679, 13383, 13397, 13399, 13398, 13391, 36912, 44076, 27476, 10891]

# Данные по определенному объекту

In [ ]:
# Для скачивания данных по определенному объекту, нужно добавить его id в конец строки запроса
url_obj = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/13383'

In [12]:
objects_df = pd.DataFrame()
for i in range(len(objids)):
    url_obj = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objids[i]}'
    try:
        res_obj = requests.get(url_obj)
        obj_data = res_obj.json()
        img_list = obj_data.get('data').get('photoRenderDTO')
        res_img = img_list[0].get('objRenderPhotoUrl')
        obj_data.get('data')['img'] = res_img
        objects_df = pd.concat([objects_df, pd.json_normalize(obj_data.get('data'), max_level=1)])
    except:
        continue
objects_df = objects_df.reset_index(drop=True)

In [13]:
pd.set_option('display.max_rows', None, 'display.max_columns', None)
objects_df.head(10)

,id,pdId,region,address,floorMin,floorMax,objElemLivingCnt,objReady100PercDt,wallMaterialShortDesc,objLkFinishTypeDesc,objLkFreePlanDesc,objElemParkingCnt,objSquareLiving,rpdNum,rpdPdfLink,rpdIssueDttm,objLkLatitude,objLkLongitude,objCompensFundFlg,objProblemFlg,objLkClassDesc,loadDttm,photoRenderDTO,objGuarantyEscrowFlg,objectType,miniUrl,residentialBuildings,newBuildingId,objFloorCnt,objFlatCnt,objFlatSq,objNonlivElemCnt,objStatus,isAvailableWantLiveHere,objTransferPlanDt,objLivCeilingHeight,objInfrstrBicycleLaneFlg,objInfrstrPlaygrndCnt,objInfrstrSportGroundCnt,objInfrstrTrashAreaCnt,objInfrstrObjPrkngCnt,objInfrstrNotObjPrkngCnt,objInfrstrRampFlg,objInfrstrCurbLoweringFlg,objElevatorWheelchairCnt,objElevatorPassengerCnt,objElevatorCargoCnt,objElevatorCargoPassengerCnt,soldOutPerc,objPriceAvg,generalContractorNm,nonlivFirstFloor,objectTransportInfo,conclusion,objLkSaleDepEmail,objGreenHouseFlg,objEnergyEffShortDesc,infrastructureIndexValue,img,developer.devId,developer.devShortCleanNm,developer.devShortNm,developer.devFullCleanNm,developer.problObjCnt,developer.buildObjCnt,developer.comissObjCnt,developer.regRegionDesc,developer.devPhoneNum,developer.devSite,developer.devEmail,developer.devInn,developer.devOgrn,developer.devKpp,developer.devLegalAddr,developer.devFactAddr,developer.lastRpdId,developer.fundGuarantyFlg,developer.devOrgRegRegionCd,developer.devEmplMainFullNm,developer.developerGroupName,developer.orgForm,developer.companyGroupId,developer.objGuarantyEscrowFlg,developer.govFundFlg,quartography.objLivElem1KCnt,quartography.objLivElem2KCnt,quartography.objLivElem3KCnt,quartography.objLivElem4KCnt,quartography.objLivElemCnt,quartography.objLivElemEntrCnt,quartography.objLivElemEntrFloorAvg,transportDistIndex,transportDistIndexValue,nameObj,greenAreaIndexValue,airQualityIndexValue,complexShortNm,developer.bankruptStage,developer.orgBankruptMsgDttm,metro.id,metro.name,metro.line,metro.color,metro.time,metro.isWalk,metro.colors
0,13383,28105,72,г Тюмень,4,20,68,2022-12-31,Другое,По желанию заказчика,Есть,31,6293.78,72-000095,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,10-10-2022 14:47,57.1519,65.5536,1,0,Элитный,14-10-2022 01:26,"[{'objId': 13383, 'pdId': 28105, 'rpdId': 6450...",0,normal,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,1,39024,20,68,6293.78,10,0,True,28-02-2023 12:00,"2,8",0,1.0,1.0,1.0,0.0,11.0,1,1,0.0,5.0,1.0,7,0.407164,53109.0,ООО Транспромжилстрой-2005 (ИНН: 7204097730),1,[],False,info@2mengroup.ru,0,B,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,Тюменская область,+7(345)279-18-88,2mengroup.ru,info@2mengroup.ru,7701651356,1067746424899,720301001,"Тюменская область, город Тюмень, улица Комсомо...","Тюменская область, город Тюмень, улица Комсомо...",645065,1,72,Киселев Сергей Михайлович,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,"{'id': 1, 'fullForm': 'Акционерное общество', ...",5776.0,0,0,13.0,21.0,34.0,0.0,68.0,1.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13397,28141,72,"г Тюмень, ул Профсоюзная, д. 56",6,12,65,2024-12-31,Другое,Без отделки,Нет,0,6248.51,72-000097,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,10-10-2022 14:49,57.1519,65.5544,0,0,Элитный,14-10-2022 01:26,"[{'objId': 13397, 'pdId': 28141, 'rpdId': 6450...",0,normal,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,1,37201,12,65,6248.51,70,0,True,28-02-2025 12:00,"2,96",0,1.0,1.0,1.0,11.0,10.0,1,1,0.0,5.0,4.0,9,0.999984,45523.0,ООО Транспромжилстрой-2005 (ИНН: 7204097730),0,[],False,info@2mengroup.ru,0,B,10,https://xn--80az8a.xn--d1aqf.xn--p1ai/api/ext/...,306,2МЕН ГРУПП,2МЕН ГРУПП,2МЕН ГРУПП,0,5,0,Тюменская область,+7(345)279-18-88,2mengroup.ru,info@2mengroup.ru,7701651356,1067746424899,720301001,"Тюменская область, город Тюмень, улица Комсомо...","Тюменская область, город Тюмень, улица Комсомо...",645065,1,72,Киселев Сергей Михайлович,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,"{'id': 1, 'fullForm': 'Акционерное общество', ...",5776.0,0,0,0.0,24.0,35.0,6.0,65.0,4.0,3.0,NaN,NaN,NaN,N

In [8]:
objects_df.to_csv('data.csv', index=0)

In [4]:
objects_df.to_json('data.json')